In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import latentplan.utils as utils
import latentplan.datasets as datasets
from latentplan.models.vqvae import VQContinuousVAE

/Users/mehmeteneserciyes/.pyenv/versions/miniconda3-latest/envs/tap/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
pybullet build time: Dec 25 2022 20:49:00


fail to lead iql


In [51]:
import os
from torch.utils.data.dataloader import DataLoader
import torch, numpy as np
import random
import numpy as np

In [91]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [79]:
class Parser(utils.Parser):
    dataset: str = 'hopper-medium-replay-v2'
    config: str = 'config.vqvae'

#######################
######## setup ########
#######################

args = Parser().parse_args('train')


env_name = args.dataset if "-v" in args.dataset else args.dataset+"-v0"
env = datasets.load_environment(env_name)

sequence_length = args.subsampled_sequence_length * args.step
args.logbase = os.path.expanduser(args.logbase)
args.savepath = os.path.expanduser(args.savepath)
if not os.path.exists(args.savepath):
    os.makedirs(args.savepath)

[ utils/setup ] Reading config: config.vqvae:hopper_medium_replay_v2
[ utils/setup ] Not using overrides | config: config.vqvae | dataset: hopper_medium_replay_v2
[ utils/setup ] Made savepath: /Users/mehmeteneserciyes/logs/hopper-medium-replay-v2/vae/vq/
[ utils/setup ] Saved args to /Users/mehmeteneserciyes/logs/hopper-medium-replay-v2/vae/vq/args.json


# Dataset

In [80]:
dataset_class = datasets.SequenceDataset

dataset_config = utils.Config(
    dataset_class,
    savepath=(args.savepath, 'data_config.pkl'),
    env=args.dataset,
    penalty=args.termination_penalty,
    sequence_length=sequence_length,
    step=args.step,
    discount=args.discount,
    disable_goal=args.disable_goal,
    normalize_raw=args.normalize,
    normalize_reward=args.normalize_reward,
    max_path_length=int(args.max_path_length),
)

dataset = dataset_config()
obs_dim = dataset.observation_dim
act_dim = dataset.action_dim
if args.task_type == "locomotion":
    transition_dim = obs_dim+act_dim+3
else:
    transition_dim = 128+act_dim+3

block_size = args.subsampled_sequence_length * transition_dim # total number of dimensionalities for a maximum length sequence (T)

print(
    f'Dataset size: {len(dataset)} | '
    f'Joined dim: {transition_dim} '
    f'(observation: {obs_dim}, action: {act_dim}) | Block size: {block_size}'
)


Config: <class 'latentplan.datasets.sequence.SequenceDataset'>
    disable_goal: False
    discount: 0.99
    env: hopper-medium-replay-v2
    max_path_length: 1000
    normalize_raw: True
    normalize_reward: True
    penalty: -100
    sequence_length: 25
    step: 1

Saved config to: /Users/mehmeteneserciyes/logs/hopper-medium-replay-v2/vae/vq/data_config.pkl

[ datasets/sequence ] Sequence length: 25 | Step: 1 | Max path length: 1000
[ datasets/sequence ] Loading... Downloading dataset: http://rail.eecs.berkeley.edu/datasets/offline_rl/gym_mujoco_v2/hopper_medium_replay-v2.hdf5 to /Users/mehmeteneserciyes/.d4rl/datasets/hopper_medium_replay-v2.hdf5


load datafile: 100%|█████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 13.92it/s]


✓
[ datasets/sequence ] Segmenting... ✓
Dataset size: 399958 | Joined dim: 17 (observation: 11, action: 3) | Block size: 425


In [85]:
transition_dim

17

# Model

In [92]:
model_config = utils.Config(
    VQContinuousVAE,
    savepath=(args.savepath, 'model_config.pkl'),
    ## discretization
    vocab_size=args.N, block_size=block_size,
    K=args.K,
    ## architecture
    n_layer=args.n_layer, n_head=args.n_head, n_embd=args.n_embd * args.n_head,
    ## dimensions
    observation_dim=obs_dim, action_dim=act_dim, transition_dim=transition_dim,
    ## loss weighting
    action_weight=args.action_weight, reward_weight=args.reward_weight, value_weight=args.value_weight,
    position_weight=args.position_weight,
    first_action_weight=args.first_action_weight,
    sum_reward_weight=args.sum_reward_weight,
    last_value_weight=args.last_value_weight,
    trajectory_embd=args.trajectory_embd,
    model=args.model,
    latent_step=args.latent_step,
    ma_update=args.ma_update,
    residual=args.residual,
    obs_shape=args.obs_shape,
    ## dropout probabilities
    embd_pdrop=args.embd_pdrop, resid_pdrop=args.resid_pdrop, attn_pdrop=args.attn_pdrop,
    bottleneck=args.bottleneck,
    masking=args.masking,
    state_conditional=args.state_conditional,
)


model = model_config()
if args.normalize:
    model.set_padding_vector(dataset.normalize_joined_single(np.zeros(model.transition_dim-1)))


Config: <class 'latentplan.models.vqvae.VQContinuousVAE'>
    K: 512
    action_dim: 3
    action_weight: 5
    attn_pdrop: 0.0
    block_size: 425
    bottleneck: pooling
    embd_pdrop: 0.0
    first_action_weight: 0
    last_value_weight: 0
    latent_step: 3
    ma_update: True
    masking: uniform
    model: VQTransformer
    n_embd: 512
    n_head: 4
    n_layer: 4
    obs_shape: [-1]
    observation_dim: 11
    position_weight: 1
    resid_pdrop: 0.0
    residual: True
    reward_weight: 1
    state_conditional: True
    sum_reward_weight: 0
    trajectory_embd: 512
    transition_dim: 17
    value_weight: 1
    vocab_size: 100

Saved config to: /Users/mehmeteneserciyes/logs/hopper-medium-replay-v2/vae/vq/model_config.pkl



## Model Params

In [39]:
650 // model.model.transition_dim

25

In [40]:
i = 0
total_param = 0
for name, param in model.named_parameters():
    i+=1
    print(i, " - ", name, ": ", param.numel())
    total_param += param.numel()
print("Total params:", total_param)


1  -  model.pos_emb :  12288
2  -  model.encoder.0.ln1.weight :  512
3  -  model.encoder.0.ln1.bias :  512
4  -  model.encoder.0.ln2.weight :  512
5  -  model.encoder.0.ln2.bias :  512
6  -  model.encoder.0.attn.key.weight :  262144
7  -  model.encoder.0.attn.key.bias :  512
8  -  model.encoder.0.attn.query.weight :  262144
9  -  model.encoder.0.attn.query.bias :  512
10  -  model.encoder.0.attn.value.weight :  262144
11  -  model.encoder.0.attn.value.bias :  512
12  -  model.encoder.0.attn.proj.weight :  262144
13  -  model.encoder.0.attn.proj.bias :  512
14  -  model.encoder.0.mlp.0.weight :  1048576
15  -  model.encoder.0.mlp.0.bias :  2048
16  -  model.encoder.0.mlp.2.weight :  1048576
17  -  model.encoder.0.mlp.2.bias :  512
18  -  model.encoder.1.ln1.weight :  512
19  -  model.encoder.1.ln1.bias :  512
20  -  model.encoder.1.ln2.weight :  512
21  -  model.encoder.1.ln2.bias :  512
22  -  model.encoder.1.attn.key.weight :  262144
23  -  model.encoder.1.attn.key.bias :  512
24  -  

# Train process

In [84]:
warmup_tokens = len(dataset) * block_size ## number of tokens seen per epoch
final_tokens = 20 * warmup_tokens
trainer_config = utils.Config(
    utils.VQTrainer,
    savepath=(args.savepath, 'trainer_config.pkl'),
    # optimization parameters
    batch_size=args.batch_size,
    learning_rate=args.learning_rate,
    betas=(0.9, 0.95),
    grad_norm_clip=1.0,
    weight_decay=0.1, # only applied on matmul weights
    # learning rate decay: linear warmup followed by cosine decay to 10% of original
    lr_decay=args.lr_decay,
    warmup_tokens=warmup_tokens,
    kl_warmup_tokens=warmup_tokens*10,
    final_tokens=final_tokens,
    ## dataloader
    num_workers=0,
    device=args.device,
)


Config: <class 'latentplan.utils.training.VQTrainer'>
    batch_size: 512
    betas: (0.9, 0.95)
    device: cpu
    final_tokens: 3399643000
    grad_norm_clip: 1.0
    kl_warmup_tokens: 1699821500
    learning_rate: 0.0002
    lr_decay: False
    num_workers: 0
    warmup_tokens: 169982150
    weight_decay: 0.1

Saved config to: /Users/mehmeteneserciyes/logs/hopper-medium-replay-v2/vae/vq/trainer_config.pkl



In [86]:
loader = DataLoader(dataset, shuffle=False, batch_size=args.batch_size)

In [87]:
optimizer = model.configure_optimizers(trainer_config)

In [88]:
batch = None
start_it = 0
end_it = 19
for it, batch in enumerate(loader):
    if it < start_it:
        continue
    elif it > end_it:
        break
    reconstructed, recon_loss, vq_loss, commit_loss = model(*batch)
    loss = (recon_loss+vq_loss+commit_loss).mean()
    print(f"Loss #{it}:", loss)
    model.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), trainer_config.grad_norm_clip)
    optimizer.step()
model.train();

Loss #0: tensor(22.1342983, grad_fn=<MeanBackward0>)
Loss #1: tensor(21.7037048, grad_fn=<MeanBackward0>)
Loss #2: tensor(21.9139156, grad_fn=<MeanBackward0>)
Loss #3: tensor(23.0029526, grad_fn=<MeanBackward0>)
Loss #4: tensor(23.2258301, grad_fn=<MeanBackward0>)
Loss #5: tensor(19.0901604, grad_fn=<MeanBackward0>)
Loss #6: tensor(14.7645531, grad_fn=<MeanBackward0>)
Loss #7: tensor(13.2855978, grad_fn=<MeanBackward0>)
Loss #8: tensor(14.1377230, grad_fn=<MeanBackward0>)
Loss #9: tensor(10.3079033, grad_fn=<MeanBackward0>)
Loss #10: tensor(8.0795202, grad_fn=<MeanBackward0>)
Loss #11: tensor(6.2094078, grad_fn=<MeanBackward0>)
Loss #12: tensor(8.4369249, grad_fn=<MeanBackward0>)
Loss #13: tensor(12.7121677, grad_fn=<MeanBackward0>)
Loss #14: tensor(14.4776773, grad_fn=<MeanBackward0>)
Loss #15: tensor(20.2491646, grad_fn=<MeanBackward0>)
Loss #16: tensor(18.9114323, grad_fn=<MeanBackward0>)
Loss #17: tensor(20.6282177, grad_fn=<MeanBackward0>)
Loss #18: tensor(17.6844177, grad_fn=<Mea